<b>Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:</b>

Наименование вакансии.<br>
Предлагаемую зарплату (отдельно минимальную и максимальную).<br>
Ссылку на саму вакансию.<br>
Сайт, откуда собрана вакансия.<br>### По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs

In [2]:
job = 'Программист'

In [3]:
def fix_salary(salary):
    
    drop = "./"
    
    before = ''
    after = ''
    min_salary = ''
    max_salary = ''
    
    for char in drop:
        salary = salary.replace(char,"")
 
    if 'от' in salary:
        after = 1   
    if 'до' in salary:
        before = 1
            
    if '–' in salary:
        min_salary, max_salary = salary.split("–")
    
    if after:
        min_salary = salary
    
    if before:
        max_salary = salary
        
    min_salary = re.sub(r'[^\x00-\x7F]+','', min_salary)
    max_salary = re.sub(r'[^\x00-\x7F]+','', max_salary)
    min_salary = min_salary.replace(' ', '')
    max_salary = max_salary.replace(' ', '')
                     
    salary = [min_salary, max_salary]
    
    return salary

In [4]:
def get_hh(text, area='1', salary=''):
    # text - название вакансии
    # area - расположение (id территории)
    # salary - зарплата
    
    url = 'https://hh.ru/search/vacancy'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
    }
    
    params = {'text': text}    
    
    if salary:
        params.update({'salary': salary, 'only_with_salary': 'true'}) 
     
    data = []
    
    html = requests.get(url, params=params, headers=headers)
    html = bs(html.text,'html.parser')
    
    pages = html.find('div', {'data-qa': 'pager-block'})
    
    if pages:
        last_page = int(pages.find_all('a', {'class': 'bloko-button'})[-2].getText())
    else:
        last_page = 1
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(url, params=params, headers=headers)
        html = bs(html.text,'html.parser')
        
        vacancy = html.find_all('div', {'class': 'vacancy-serp-item'})
        for item in vacancy:
            title = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).getText()
            salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            salary = fix_salary(salary.get_text()) if salary else ['','']
            link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
            site = 'hh.ru'
            this = ({'title': title, 'min_salary': salary[0], 'max_salary': salary[1], 'link': link, 'site': site}) 
            data.append(this)
       
    return data

In [5]:
def get_sj(text, area='', salary=''):
    
    url = 'https://www.superjob.ru/vacancy/search'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
    }
    
    params = {'keywords': text}    
    if salary:
        params.update({'payment_value': salary, 'payment_defined': '1'}) 
     
    data = []
    
    html = requests.get(url, params=params, headers=headers)
    html = bs(html.text,'html.parser')
    
    pages = html.find('a', {'class': 'f-test-button-1'})
    
    if pages:
        pages = pages.findParent()
        last_page = int(pages.find_all('a')[-2].getText())
    else:
        last_page = 1
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(url, params=params, headers=headers)
        html = bs(html.text,'html.parser')
        
        vacancy = html.find_all('div', {'class': 'f-test-vacancy-item'})
        for item in vacancy:
            a = item.find_all('div')
            title = item.find('a', {'target': '_blank'}).getText()
            salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).getText()
            site = 'superjob.ru'
            link = f"http://{site}{item.find('a', {'target': '_blank'})['href']}"
            
            salary = fix_salary(salary)
            this = ({'title': title, 'min_salary': salary[0], 'max_salary': salary[1], 'link': link, 'site': site}) 
            data.append(this)
            
        return data 

In [6]:
hh = get_hh(job)
sj = get_sj(job)
all = hh+sj

In [11]:
df = pd.DataFrame(all)
df.head(1000)

,title,min_salary,max_salary,link,site
0,Программист-разработчик frontend,210000,210000,https://hh.ru/vacancy/48722695?from=vacancy_se...,hh.ru
1,Программист,,,https://hh.ru/vacancy/39425130?from=vacancy_se...,hh.ru
2,Ведущий программист 1С (вулиця Січових Стрільц...,,,https://hh.ru/vacancy/48715321?from=vacancy_se...,hh.ru
3,Программист 1С,10000000,,https://hh.ru/vacancy/48264925?from=vacancy_se...,hh.ru
4,"Ведущий программист 1C (Пироговский шлях, 34)",,,https://hh.ru/vacancy/48716975?from=vacancy_se...,hh.ru
...,...,...,...,...,...
815,PHP-разработчик,,,http://superjob.ru/vakansii/php-razrabotchik-3...,superjob.ru
816,Разработчик SQL,,,http://superjob.ru/vakansii/razrabotchik-sql-3...,superjob.ru
817,Junior разработчик Ruby on Rails,,,http://superjob.ru/vakansii/junior-razrabotchi...,superjob.ru
818,"Frontend разработчик (js, Vue)",,,http://superjob.ru/vakansii/frontend-razrabotc...,superjob.ru
